<a href="https://colab.research.google.com/github/SASHI117/Plant-Disease-Classification/blob/main/plant_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras gradio

In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sashi4080","key":"4effaf2d354d69941712584770df63e5"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d emmarex/plantdisease
!unzip plantdisease.zip -d plant_disease

Streaming output truncated to the last 5000 lines.
  inflating: plant_disease/plantvillage/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/08c033bd-fbc3-445a-88d1-1863070e52ce___YLCV_GCREC 2872.JPG  
  inflating: plant_disease/plantvillage/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/08dd176c-e9d9-4746-92c3-fa8dc9074347___UF.GRC_YLCV_Lab 03057.JPG  
  inflating: plant_disease/plantvillage/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/08f78a80-46f5-45a6-937c-4d05d61c08c2___UF.GRC_YLCV_Lab 01895.JPG  
  inflating: plant_disease/plantvillage/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/0903aa95-6e8a-4abd-a003-126fcd9a5493___YLCV_GCREC 2806.JPG  
  inflating: plant_disease/plantvillage/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/0911d416-d73d-4c2a-8e45-207a7ceb7c9a___YLCV_GCREC 2773.JPG  
  inflating: plant_disease/plantvillage/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/09188838-cc89-4925-94c5-d9563c4cb4bf___UF.GRC_YLCV_Lab 02974.JPG  
  inflating: plant_

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import gradio as gr
import numpy as np


In [ ]:
train_dir = "/content/plant_disease/PlantVillage"
valid_dir = "/content/plant_disease/PlantVillage"

img_size = 160
batch_size = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data for validation
)

valid_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

valid_data = valid_gen.flow_from_directory(
    valid_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.


In [ ]:
!ls -R /content/plant_disease

Streaming output truncated to the last 5000 lines.
'ff20d775-59a6-437c-9039-f72d07aa4484___Com.G_SpM_FL 8524.JPG'
'ff6346ca-227b-4cb4-a350-0941048744fa___Com.G_SpM_FL 9338.JPG'
'ff9a52b5-b200-4f23-9e0d-629b1a0e89db___Com.G_SpM_FL 8634.JPG'
'ffb00771-1462-4c9d-962b-7aaa7e8d5a51___Com.G_SpM_FL 1411.JPG'
'ffb50767-1893-4379-ba42-8af6d8030057___Com.G_SpM_FL 8854.JPG'
'ffcfa32d-4506-49ed-9e35-972d5cbad0b8___Com.G_SpM_FL 8450.JPG'
'fff3aeaf-d4ae-4f3a-86d9-79237d07270f___Com.G_SpM_FL 9349.JPG'
'fff8949b-3721-4f42-a489-8d39753fe7bb___Com.G_SpM_FL 8656.JPG'

/content/plant_disease/PlantVillage/Tomato__Target_Spot:
'002213fb-b620-4593-b9ac-6a6cc119b100___Com.G_TgS_FL 8360.JPG'
'003a5321-0430-42dd-a38d-30ac4563f4ba___Com.G_TgS_FL 8121.JPG'
'0044d10f-7062-4655-8abd-9fffc5b2f152___Com.G_TgS_FL 7908.JPG'
'007ec913-8cfe-43ed-b458-f041a17266ae___Com.G_TgS_FL 9844.JPG'
'00854623-ba28-446d-a1d5-03c0ae3f99d9___Com.G_TgS_FL 7949.JPG'
'0118c27a-f34a-41e1-8b2f-d2c564f83c5c___Com.G_TgS_FL 0013.JPG'
'0118ed61

In [ ]:
base = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))

# Unfreeze last 40 layers → more trainable params + higher accuracy
for layer in base.layers[:-40]:
    layer.trainable = False
for layer in base.layers[-40:]:
    layer.trainable = True

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base.input, outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 80, 80,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 80, 80,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 80, 80,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 80, 80,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 80, 80,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 80, 80,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 81, 81,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 40, 40,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 40, 40,    │      2,304 │ block_1_depthwis

 Total params: 2,277,199 (8.69 MB)

 Trainable params: 1,700,751 (6.49 MB)

 Non-trainable params: 576,448 (2.20 MB)

In [ ]:
!ls -R /content/plant_disease


Streaming output truncated to the last 5000 lines.
'ff20d775-59a6-437c-9039-f72d07aa4484___Com.G_SpM_FL 8524.JPG'
'ff6346ca-227b-4cb4-a350-0941048744fa___Com.G_SpM_FL 9338.JPG'
'ff9a52b5-b200-4f23-9e0d-629b1a0e89db___Com.G_SpM_FL 8634.JPG'
'ffb00771-1462-4c9d-962b-7aaa7e8d5a51___Com.G_SpM_FL 1411.JPG'
'ffb50767-1893-4379-ba42-8af6d8030057___Com.G_SpM_FL 8854.JPG'
'ffcfa32d-4506-49ed-9e35-972d5cbad0b8___Com.G_SpM_FL 8450.JPG'
'fff3aeaf-d4ae-4f3a-86d9-79237d07270f___Com.G_SpM_FL 9349.JPG'
'fff8949b-3721-4f42-a489-8d39753fe7bb___Com.G_SpM_FL 8656.JPG'

/content/plant_disease/PlantVillage/Tomato__Target_Spot:
'002213fb-b620-4593-b9ac-6a6cc119b100___Com.G_TgS_FL 8360.JPG'
'003a5321-0430-42dd-a38d-30ac4563f4ba___Com.G_TgS_FL 8121.JPG'
'0044d10f-7062-4655-8abd-9fffc5b2f152___Com.G_TgS_FL 7908.JPG'
'007ec913-8cfe-43ed-b458-f041a17266ae___Com.G_TgS_FL 9844.JPG'
'00854623-ba28-446d-a1d5-03c0ae3f99d9___Com.G_TgS_FL 7949.JPG'
'0118c27a-f34a-41e1-8b2f-d2c564f83c5c___Com.G_TgS_FL 0013.JPG'
'0118ed61

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_path = "/content/plant_disease/PlantVillage"
train_path = "/content/plant_disease/train"
valid_path = "/content/plant_disease/valid"

# Create train & valid folders if not exists
os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)

# Loop classes
for cls in os.listdir(base_path):

    cls_path = os.path.join(base_path, cls)
    if not os.path.isdir(cls_path):
        continue

    images = os.listdir(cls_path)
    train_imgs, valid_imgs = train_test_split(images, test_size=0.2, random_state=42)

    # Make class folders
    os.makedirs(os.path.join(train_path, cls), exist_ok=True)
    os.makedirs(os.path.join(valid_path, cls), exist_ok=True)

    # Copy images
    for img in train_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(train_path, cls))

    for img in valid_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(valid_path, cls))

print("Dataset split completed!")
!ls -R /content/plant_disease


Streaming output truncated to the last 5000 lines.
'67727242-9c27-4ab5-8b7b-2ae2a33080c8___YLCV_NREC 2750.JPG'
'6781756b-f3be-4449-a1c6-ac0a50ae0bf5___YLCV_GCREC 1994.JPG'
'67827bc5-e033-4e96-ae21-12c06dffa03f___YLCV_GCREC 5300.JPG'
'67905f2f-163a-4abf-8306-73ab9515af8c___YLCV_GCREC 2162.JPG'
'67907001-4326-4dcf-b60a-d9a59ffeb767___YLCV_GCREC 2167.JPG'
'67b7c85c-f6e2-419f-b3a7-20bcae8c3afa___UF.GRC_YLCV_Lab 01429.JPG'
'67be104d-cbf0-4ec8-9137-6d3cc1f2b9ed___YLCV_GCREC 5330.JPG'
'67d2fd78-d897-4e0c-831c-e8d373829a9f___YLCV_GCREC 2198.JPG'
'67d8a1e8-668e-40ad-a8d2-3da621594ecf___YLCV_GCREC 2598.JPG'
'67e44e1c-6554-4a77-8565-b174f66c3ff9___UF.GRC_YLCV_Lab 01412.JPG'
'67e78746-5c79-43b8-a73f-3ba6cb0b91f9___UF.GRC_YLCV_Lab 02823.JPG'
'680826bf-080d-4fc9-9148-2bb2398b4503___UF.GRC_YLCV_Lab 01290.JPG'
'681e6fac-850e-44c7-ba3c-cbee5831fc2e___YLCV_GCREC 1972.JPG'
'68243147-87d2-4a8e-96ba-bf5388d2ea51___YLCV_GCREC 5466.JPG'
'6824b68a-5483-4a36-ab65-c1fed7a153a6___YLCV_GCREC 5164.JPG'
'685faff6-c

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Paths
train_dir = '/content/plant_disease/train'
valid_dir = '/content/plant_disease/valid'

# Image settings (smaller image speeds up training)
img_size = 160
batch = 32

# Data Generators
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
)

valid_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch,
    class_mode='categorical'
)

valid_data = valid_gen.flow_from_directory(
    valid_dir,
    target_size=(img_size, img_size),
    batch_size=batch,
    class_mode='categorical'
)

# Load MobileNetV2
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(img_size, img_size, 3)
)

# Freeze all layers first
base_model.trainable = False

# Build model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(base_model.input, output)

# First compile (fast training)
model.compile(
    optimizer=Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 1 → Train head only (fast)
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=3
)

# Step 2 → Fine-tune last 40 layers (faster than full unfreeze)
for layer in base_model.layers[-40:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 2 training
history_fine = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=4   # Total = 7 epochs
)

# Save model
model.save("plant_model_fast.h5")
print("MODEL TRAINED FAST & SAVED!")


Found 16504 images belonging to 15 classes.
Found 4134 images belonging to 15 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
516/516 ━━━━━━━━━━━━━━━━━━━━ 525s 1s/step - accuracy: 0.6564 - loss: 1.0874 - val_accuracy: 0.8706 - val_loss: 0.3962
Epoch 2/3
516/516 ━━━━━━━━━━━━━━━━━━━━ 513s 995ms/step - accuracy: 0.8333 - loss: 0.4837 - val_accuracy: 0.8832 - val_loss: 0.3525
Epoch 3/3
516/516 ━━━━━━━━━━━━━━━━━━━━ 516s 1s/step - accuracy: 0.8632 - loss: 0.4075 - val_accuracy: 0.8817 - val_loss: 0.3381
Epoch 1/4
516/516 ━━━━━━━━━━━━━━━━━━━━ 714s 1s/step - accuracy: 0.8227 - loss: 0.5709 - val_accuracy: 0.7344 - val_loss: 1.0445
Epoch 2/4
516/516 ━━━━━━━━━━━━━━━━━━━━ 699s 1s/step - accuracy: 0.9244 - loss: 0.2145 - val_accuracy: 0.8065 - val_loss: 0.7032
Epoch 3/4
516/516 ━━━━━━━━━━━━━━━━━━━━ 695s 1s/step - accuracy: 0.9402 - loss: 0.1752 - val_accuracy: 0.9149 - val_loss: 0.2572
Epoch 4/4
516/516 ━━━━━━━━━━━━━━━━━━━━ 690s 1s/step - accuracy: 0.9561 - loss: 0.1326 - val_accuracy: 0.9434 - val_loss: 0.1729


MODEL TRAINED FAST & SAVED!


In [ ]:
!pip install gradio


In [ ]:
model = tf.keras.models.load_model("plant_model_fast.h5")


In [ ]:
model = tf.keras.models.load_model("plant_model_fast.h5")

# OPTIONAL — to remove warning
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
model = tf.keras.models.load_model("plant_model_fast.h5")


In [ ]:
import tensorflow as tf
import numpy as np
import gradio as gr
from tensorflow.keras.preprocessing import image

# Load trained model
model = tf.keras.models.load_model("plant_model_fast.h5")

# Same 15 classes used during training
class_names = sorted([
    d for d in os.listdir("/content/plant_disease/train")
    if os.path.isdir(os.path.join("/content/plant_disease/train", d))
])

# Prediction function
def predict(img):
    img = img.resize((160, 160))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)
    class_index = np.argmax(pred[0])
    confidence = float(np.max(pred[0]) * 100)

    return {class_names[class_index]: confidence}


In [ ]:
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="🌱 Plant Disease Classifier",
    description="Upload a leaf image to detect the plant disease."
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2aca98d2ff6380422c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
model.save('/content/drive/MyDrive/plant_model_fast.h5')


In [ ]:
!find / -name "plant_model_fast.h5" 2>/dev/null

/content/plant_model_fast.h5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model.save('/content/drive/MyDrive/plant_model_fast.h5')


In [ ]:
!ls -lh /content/drive/MyDrive


total 96M
-rw------- 1 root root  183 Oct 15 13:21 '007_AmithKumar _OfferLetterMatrical.gdoc'
-rw------- 1 root root  183 May  5  2025  1000070476.gdoc
-rw------- 1 root root 6.8M Aug 25  2019  10th-Physics-EM-PDFs.rar
-rw------- 1 root root  83K Dec 26  2024 '27 Dec Cocubes Assessment Lab Wise student Data.xlsx'
-rw------- 1 root root 7.0M May 26 17:18  79301.mp4
-rw------- 1 root root 811K Jul  1  2024  971706600012013_signed.pdf
-rw------- 1 root root  183 Apr  6  2025 'AAD CASE STUDY B1.gdoc'
-rw------- 1 root root  183 Nov  9 11:48 'A C Details.gsheet'
-rw------- 1 root root  183 Sep 24 04:10 'Airport Authority of India.gslides'
-rw------- 1 root root  183 Jan 16  2025 'Annual budget.gsheet'
-rw------- 1 root root  183 Sep 13  2024 'Answers FM.gdoc'
-rw------- 1 root root 148K Sep 13  2024 'Answers FM.pdf'
-rw------- 1 root root  183 May 13  2025 'as1-2nd question.gdoc'
-rw------- 1 root root  183 May  7  2023 'Book report.gdoc'
-rw------- 1 root root  183 Nov  4  2024 'Book repor